In [ ]:
###############     FULLY CONNECTED DEEP NETWORK
import torch
import torch.nn as nn
import torchvisiontorchvision.transforms

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 784  #####   For Mnist


In [ ]:
# Download data, if not already at  '\root' location

train_dataset = torchvision.datasets.MNIST(root='../datasets/', train = True,
                                          transform=torchvision.transforms.ToTensor(),
                                          download = True)
test_dataset = torchvision.datasets.MNIST(root='../datasets/', train = False,
                                          transform=torchvision.transforms.ToTensor(),
                                          download = True)
#  Create iterators

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=32,
                                          shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=32,
                                          shuffle = False)


In [ ]:
num_classes = 10
n_epochs = 25
steps = len(train_loader)
hidden = 500    #######    for 1 hidden layer
class Net(nn.Module):
    
    def __init__(self, input_size, hidden, num_classes):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_size, hidden)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden,num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = Net(input_size, hidden, num_classes).to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

for e in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images  = images.reshape(-1,784).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss_val = loss(outputs, labels)
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()
        
        if (e+1)%5==0:
            print('Epoch [{}/{}], step [{}/{}], loss-----> {:.3f}'.format(
                e+1,n_epochs, i, steps, loss_val.item()))
        
        

In [ ]:
with torch.no_grad():
    right = 0
    total = 0
    for images, labels in test_loader:
        images  = images.reshape(-1,784).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, pred = torch.max(outputs.data, 1)
        total+= labels.size(0)
        right+=(pred==labels).sum().item()
    print('Test Accuracy  .....> {:.3f}'.format(100*right/total))

    torch.save(model.state_dict(), '../datasets/models/torch103.ckpt')
        